# Chapter 8 데이터 준비하기 : 조인, 병합, 변형

## 8.1 계층적 색인
계층적 색인 : 축에 대해 둘 이상의 색인 단계 지정

In [4]:
import pandas as pd
import numpy as np

In [5]:
data=pd.Series(np.random.uniform(size=9),
               index=[["a","a","a","b","b","c",'c',"d","d"],
                      [1,2,3,1,3,1,2,2,3]])

In [6]:
data

a  1    0.642541
   2    0.013136
   3    0.009842
b  1    0.655074
   3    0.600430
c  1    0.328447
   2    0.459318
d  2    0.940126
   3    0.860849
dtype: float64

In [8]:
# MultiIndex를 색인으로 한는 계층 확인
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [9]:
# 부분적 색인으로 접근
data["b"]

1    0.655074
3    0.600430
dtype: float64

In [10]:
data["b":"c"]

b  1    0.655074
   3    0.600430
c  1    0.328447
   2    0.459318
dtype: float64

In [11]:
# 하위 계층의 객체로 선택 가능

data.loc[:,2]
# 하위 계층의 색인이 2인 모든 값

a    0.013136
c    0.459318
d    0.940126
dtype: float64

In [12]:
# 데이터를 재구성하고 피벗 테이블 생성 같은 그룹 기반으로 작업 가능
#unstack 메서드를 사용해 새롭게 배열
data.unstack()

# 반대는 stack()

,1,2,3
a,0.642541,0.013136,0.009842
b,0.655074,NaN,0.600430
c,0.328447,0.459318,NaN
d,NaN,0.940126,0.860849


In [17]:
# 행과 열 모두 계층적 색인 가능
frame=pd.DataFrame(np.arange(12).reshape((4,3)),
                   index=[["a","a","b","b"],[1,2,1,2]],
                   columns=[["Ohio","Ohio","Colorado"],
                   ["Green","Red","Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [23]:
# 계측정 색인의 각 단계에 이름을 설정하고 출력
frame.index.names=["key1","key2"]
frame.columns.names=["state","color"]

frame

# 주의할 점 : 행 레이블과 색인 이름을 혼동하지 않을 것.

2

In [24]:
pd.MultiIndex.from_arrays([["Ohio","Ohio","Colorado"],
                           ["Green","Red","Green"]],
                          names=["state","color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### 8.1.1 계층의 순서를 바꾸고 정렬하기

In [25]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [28]:
# 두 개의 계층을 바꾼 새로운 객체 반환
frame.swaplevel("key1","key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [29]:
# level 인수를 이용해 단일계층만 사용하거나 일부 계츠만 선택해 정렬
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [34]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 계층별 요약 통계

In [35]:
# 하나의 축에 대해 합을 계층별로 계산
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [36]:
frame.groupby(level="color",axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 DataFrame의 열 사용하기

In [38]:
frame=pd.DataFrame({"a":range(7), "b" : range(7,0,-1),
                    "c":["one","one","one","two","two","two","two"],
                    "d":[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [39]:
# 하나 이상의 열을 색인으로 하는 새로운 DataFrame 생성
frame2=frame.set_index(["c","d"])
frame2

# drop=False를 명시적으로 지정하지 않으면 색인으로 지정한 열은 삭제

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [41]:
# reset_index 함수는 set_index와 반대, 계층적 색인 단계가 열로 이동
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 데이터 합치기

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [43]:
# pandas.merge 함수를 이용해 행 연결하기

df1=pd.DataFrame({"key":["b","b","a","c","a","a","b"],
                  "data1":pd.Series(range(7),dtype="Int64")})
df2=pd.DataFrame({"key":["a","b","d"],
                  "data2":pd.Series(range(3),dtype="Int64")})

In [44]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [47]:
# 겹치는 열의 이름으로 병합
pd.merge(df1,df2,on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [48]:
# 다대다 병합은 곱집합을 생성
df1=pd.DataFrame({"key":["b","b","a","c","a","a"],
                  "data1":pd.Series(range(6),dtype="Int64")})
df2=pd.DataFrame({"key":["a","b","a","b","d"],
                  "data2":pd.Series(range(5),dtype="Int64")})

In [49]:
pd.merge(df1,df2,on="key",how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,a,5,0


In [51]:
pd.merge(df1,df2,how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,a,5,0
9,a,5,2


In [53]:
left=frame=pd.DataFrame({"key1":["foo","foo","bar"],
                    "key2":["one","two","one"],
                    "lval":pd.Series([1,2,3],dtype='Int64')})

right=frame=pd.DataFrame({"key1":["foo","foo","bar","bar"],
                    "key2":["one","one","one","two"],
                    "lval":pd.Series([4,5,6,7],dtype='Int64')})

pd.merge(left,right,on=["key1","key2"],how="outer")

,key1,key2,lval_x,lval_y
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [56]:
# 겹치는 열 이름 처리
# suffixes 옵션으로 겹친는 열 이름 뒤에 문자열 지정

pd.merge(left,right,on="key1",suffixes=("_left","_right"))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 색인 병합하기

In [62]:
# 병합하려는 키가 색인행 레이블일 경우에는 right/left_index=True 옵션으로 병합 가능

left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

pd.merge(left1, right1, left_on="key", right_index=True)

In [66]:
#외부조인 합집합
pd.merge(left1,right1,left_on="key",right_index=True,how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [68]:
#계츠적으로 색인된 데이터 병합
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})


In [69]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [70]:
# 양쪽에 공통으로 존재하는 여러 개의 색인 병합
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [71]:
# 색인으로 병합할 때의 join 메서드 : 열이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 갖는 DataFrame 객체 병합

# 기본적으로 왼쪽 조인 수행
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [72]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### 8.2.3 축 따라 이어 붙이기

In [73]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [74]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [75]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [76]:
# 개별 series 결과를 구분할 수 있도록 계측정 색인을 생성하여 식별
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [77]:
# axis="columns"으로 병합하면 keys는 DataFrame의 열 제목
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [78]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [79]:
# 객체의 딕셔너리를 넘기면 keys 옵션으로 딕셔너리 키 사용
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [80]:
# 계층적 색인 생성할 때는 names 인수 지정
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [81]:
# 행 색인이 분석에 필요한 데이터를 포함하지 않는 경우

df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])

In [82]:
# ignore_index=True 옵션으로 각 DataFrame의 색인 무시, 열에 있는 데이터만 이어 붙여 기본 색인 할당
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-1.001379,2.302621,-1.427253,0.604333
1,-0.605452,-0.516634,1.200948,-0.692619
2,-0.046646,-0.464458,0.183860,0.331908
3,-0.949907,-0.940147,NaN,0.688125
4,-0.504682,-0.526649,NaN,-0.825476


### 8.2.4 겹치는 데이터 합치기

두 데이터셋의 색인 중 일부가 겹치거나 전체가 겹치는 경우 -> 병합 불가  
넘파이의 where 함수 이용

In [83]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])

In [84]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

a 값이 null일 때 b가 선택, 그렇지 않으면 a 선택  
numpy.where 사용하면 색인 레이블의 정렬 여부 확인 하지 않음.

In [85]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})

In [86]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.2 재구성과 피벗

### 8.3.1 계층적 색인으로 재구성하기

In [87]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [90]:
# stack : 데이터의 열을 행으로 피벗한다.
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [89]:
# unstack : 행을 열로 피벗한다.
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


### 8.3.2 긴 형식에서 넓은 형식으로 피벗하기

In [92]:
# https://github.com/statsmodels/statsmodels/blob/main/statsmodels/datasets/macrodata/macrodata.csv

data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

SyntaxError: ignored

In [93]:
# pandas.PeriodIndex : 시간 간격을 나타냄.
# 연도와 분기 열을 합치고 각 분기 말에 날자/시간 값을 포함하도록 색인 설정

periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

KeyError: ignored

In [94]:
data.index = periods.to_timestamp("D")
data.head()

NameError: ignored

In [95]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
state,,,
Ohio,NaN,NaN,NaN
Colorado,NaN,NaN,NaN


In [96]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))